In [1]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

import pathlib
import shutil

def ontem(hoje):
    distD = int(hoje.strftime('%d'))-1
    distM = int(hoje.strftime('%m'))
    distA = 21
    if distD < 10:
        return '0{}/0{}/{}'.format(str(distD),str(distM),str(distA))
    return '{}/0{}/{}'.format(str(distD),str(distM),str(distA))

def dia_de_anteontem(hoje):
    dist = int(hoje.strftime('%d'))-2
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)


def download_wait(directory, timeout, nfiles=None):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

def download_check(directory, file_name):
    seconds = 0
    no_dl = True
    files = os.listdir(directory)
    for fname in files:
        if fname.endswith('{}'.format(file_name)):
            return True
    raise ValueError("Nao está")

lista=['isolamento','leitos_ocupados_por_unidade_hospitalar','vacinometro']
home = Path.home()
parent = pathlib.Path().absolute()
d = Path(home,'Downloads')
cidade='piracicaba'
t=Path(parent,"bases/{}/".format(cidade))
source_dir = d
target_dir = t
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")


hoje = datetime.now()
dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
danteontem = dia_de_anteontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')
noite=False
for item in lista:
    driver = webdriver.Chrome(path_win)
    path_download = Path(parent,"bases/{}/{}.csv".format(cidade,item))
    
    link1 = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dhoje,item)
    link2= "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_{}.csv".format(mes,mes,dontem,item)
        

    t1=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dhoje,item))
    t2=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,dontem,item))
    t3=Path(parent,"bases/{}/2021{}{}_{}.csv".format(cidade,mes,danteontem,item))

    d1=Path(d,"2021{}{}_{}.csv".format(mes,dhoje,item))
    d11=Path(d,"2021{}{}_{} (1).csv".format(mes,dhoje,item))
    d2=Path(d,"2021{}{}_{}.csv".format(mes,dontem,item))
    d21=Path(d,"2021{}{}_{} (1).csv".format(mes,dontem,item))
    d3=Path(d,"2021{}{}_{}.csv".format(mes,danteontem,item))
    d31=Path(d,"2021{}{}_{} (1).csv".format(mes,danteontem,item))


    arq1 = "2021{}{}_{}.csv".format(mes,dhoje,item)
    arq2 = "2021{}{}_{}.csv".format(mes,dontem,item)
    arq3 = "2021{}{}_{}.csv".format(mes,danteontem,item)
    
    try:
        noite=True
        data = hoje.strftime('%d/%m/%y')
        if (item == 'isolamento'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'isolamento.csv')
            driver.quit()
        elif (item == 'vacinometro'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'vacinometro.csv')
            driver.quit()
        elif (item == 'leitos_ocupados_por_unidade_hospitalar'):
            driver.get(link1)
            driver.implicitly_wait(10)
            download_wait(d,130)
            download_check(d,'leitos_ocupados_por_unidade_hospitalar.csv')
            driver.quit()
        else:
            raise ValueError("Nao está")

    except:
        noite=False
        data = ontem(hoje)
        if (item == 'isolamento'):
            driver.get(link2)
            driver.implicitly_wait(10)
            download_wait(d,130)
            driver.quit()
        else:
            driver.get(link2)
            driver.implicitly_wait(10)
            download_wait(d,30)
            driver.quit()
    

    if path_download.is_file():
        vacinas=pd.read_csv(path_download,',')
    else:
        vacinas=pd.DataFrame()
    file_names = os.listdir(source_dir)
    for file_name in file_names:
        if noite:
            if (file_name==arq1) and not(t1.is_file()):
                shutil.move(os.path.join(source_dir, file_name), target_dir)
        else:
            if file_name==arq2 and not(t2.is_file()):
                shutil.move(os.path.join(source_dir, file_name), target_dir)
            continue

    if (t2.is_file())&(t1.is_file())& noite :
        os.remove(t2)

    elif (t3.is_file()) & (t2.is_file()):
        os.remove(t3)

    if d1.is_file():
        os.remove(d1)
    if d2.is_file():
        os.remove(d2)
    if d3.is_file():
        os.remove(d3)
    if d11.is_file():
        os.remove(d11)
    if d21.is_file():
        os.remove(d21)
    if d31.is_file():
        os.remove(d31)
        
    if noite:
        try:
            df = pd.read_csv(t1,';')
        except FileNotFoundError:
            df = pd.read_csv(t2,';')
    else:
        df = pd.read_csv(t2,';')
    if item=='leitos_ocupados_por_unidade_hospitalar':
        try:
            df = df[df['Município'] == 'PIRACICABA']
            df['Data'] = data
        except:
            df = df[df.iloc[:,1] == 'PIRACICABA']
            df['Data'] = data
    elif item=='isolamento':
        try:
            df = df[df['Município1'] == 'PIRACICABA']
        except:
            df = df = df[df.iloc[:,0] == 'PIRACICABA']    
    elif item=='vacinometro':
        try:
            df = df[df['Municipio'] == 'PIRACICABA']
            df['Data'] = data
        except:
            df = df = df[df.iloc[:,0] == 'PIRACICABA']        
    else:
        continue
        
    if (path_download.is_file()):
        if vacinas['Data'].iloc[-1]!= data:
            vacinas = vacinas.append(df,ignore_index=True)
    else:
        vacinas = vacinas.append(df,ignore_index=True)
    vacinas.to_csv(path_download,index=False)


In [32]:

from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime
import pathlib
import shutil

def download_wait(directory, timeout, nfiles=None):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

def download_check(directory, file_name):
    seconds = 0
    no_dl = True
    files = os.listdir(directory)
    for fname in files:
        if fname.endswith('{}'.format(file_name)):
            return True
    raise ValueError("Nao está")
    
cidade='piracicaba'
home = Path.home()
d = Path(home,'Downloads')
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")





vacinas = pd.DataFrame()
datas=[('15','02'),('16','02'),('17','02'),('18','02'),('18','02'),('19','02'),('20','02'),('21','02'),
       ('22','02'),('23','02'),('24','02'),('25','02'),('26','02'),('27','02'),
       ('01','03'),('02','03')]
for dia,mes in datas:
    data_str='{}/{}/21'.format(dia,mes)
    data=datetime.strptime(data_str, '%d/%m/%y')
    driver = webdriver.Chrome(path_win)
    d1=Path(d,"2021{}{}_vacinometro.csv".format(mes,dia))
    link1 = "https://www.saopaulo.sp.gov.br/wp-content/uploads/2021/{}/2021{}{}_vacinometro.csv".format(mes,mes,dia)
    driver.get(link1)
    driver.implicitly_wait(10)
    download_wait(d,130)
    
    driver.quit()
    df = pd.read_csv(d1,';')
    try:
        df = df[df['Municipio'] == 'PIRACICABA']
        df['Data'] = data
    except:
        df = df[df.iloc[:,0] == 'PIRACICABA']
    vacinas = vacinas.append(df,ignore_index=True)
parent = pathlib.Path().absolute()
path_download19 = Path(parent,"bases/{}/vacinas.csv".format(cidade))
vacinas.to_csv(path_download19)

In [48]:
vacinas=pd.read_csv(path_download19)
soma_vac = vacinas.groupby('Data').sum()
soma_vac.sort_values('Data',inplace=True)

vac=soma_vac['Contagem de Id Vacinacao'].tolist()
data=soma_vac.index

pdose=vacinas.iloc[-1,2]
sdose=vacinas.iloc[-2,2]
sumvac=pdose+sdose
if sdose<10000:
    rpdose=str(pdose)[:2]
    rsdose=str(sdose)[:1]
    rsumvac=str(sumvac)[:2]
else:
    rpdose=str(pdose)[:2]
    rsdose=str(sdose)[:2]
    rsumvac=str(sumvac)[:2]


In [54]:
pdose

17024

In [3]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

import pathlib
cidade='piracicaba'
path_download15 = Path(parent,"bases/{}/covid.csv".format(cidade))

def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return '0'+str(dist)
    else:
        return str(dist)

hoje = datetime.now()
dhoje = hoje.strftime('%d')
dontem = dia_de_ontem(hoje)
mes= hoje.strftime('%m')
hora = hoje.strftime('%H')

covid = pd.read_csv(path_download15)
covid.drop('Unnamed: 0',axis=1,inplace=True)
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)
if int(hora) >18:
    dias = ['{}'.format(dhoje)]
    meses =['{}'.format(mes)]
else:
    dias = ['{}'.format(dontem)]
    meses =['{}'.format(mes)]
for m in meses:
    for d in dias:
        link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2021.aspx'.format(d,m)
        print(link)
        driver.get(link)
        div = driver.find_element_by_id('imagenet-conteudo')
        novos_obitos = 'NA'
        casos_confirmados = 'NA'
        casos_suspeitos = 'NA'
        casos_descartados = 'NA'
        casos_recuperados = 'NA'
        em_tratamento = 'NA'
        obitos = 'NA'
        homens = 'NA'
        mulheres = 'NA'
        for i in range(len(div.text.split('\n'))):
            if (len(div.text.split('\n')[i].split())<5)&(len(div.text.split('\n')[i].split())>1):
                if div.text.split('\n')[i].split()[-1] =='confirmados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_confirmados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_confirmados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='suspeitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_suspeitos = div.text.split('\n')[i].split()[0][:4]
                    else:
                        casos_suspeitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='descartados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_descartados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_descartados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='recuperados':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        casos_recuperados = div.text.split('\n')[i].split()[0][:6]
                    else:
                        casos_recuperados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='tratamento':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        em_tratamento = div.text.split('\n')[i].split()[0][:4]
                    else:
                        em_tratamento = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbitos':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbito':
                    if len(div.text.split('\n')[i].split()[0])>5:
                        obitos = div.text.split('\n')[i].split()[0][:3]
                    else:
                        obitos = div.text.split('\n')[i].split()[0]
            elif len(div.text.split('\n')[i].split())>1:
                if div.text.split('\n')[i].split()[1] =='homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher':
                    mulheres = div.text.split('\n')[i].split()[0]
        dicionario ={'Total Casos Confirmados': int(casos_confirmados.replace('.','')),
                     'Total Casos Suspeitos':int(casos_suspeitos.replace('.','')),
                    'Total Casos Descartados':int(casos_descartados.replace('.','')),
                    'Total Casos Recuperados':int(casos_recuperados.replace('.','')),
                    'Total Casos em Tratamento':int(em_tratamento.replace('.','')),
                     'Total Óbitos':int(obitos.replace('.','')),
                     'Homens':int(homens.replace('.','')),
                     'Mulheres':int(mulheres.replace('.','')),'Data':'{}/{}/2021'.format(d,m)}
        if covid['Data'].iloc[-1]!=dicionario['Data']:
            covid = covid.append(dicionario, ignore_index=True)     
            covid['Óbitos Diários'].iloc[-1]=covid['Total Óbitos'].iloc[-1]-covid['Total Óbitos'].iloc[-2]
            covid['Casos Diários'].iloc[-1]=covid['Total Casos Confirmados'].iloc[-1]-covid['Total Casos Confirmados'].iloc[-2]

        covid['Média Móvel Casos'] = covid['Casos Diários'].rolling(7).mean()
        covid['Média Móvel Óbitos'] = covid['Óbitos Diários'].rolling(7).mean()
        covid['Média Móvel Tratamento'] = covid['Total Casos em Tratamento'].rolling(7).mean()
        covid['Média Móvel Suspeitos'] = covid['Total Casos Suspeitos'].rolling(7).mean()
        
covid.to_csv(path_download15)
driver.quit()

http://www.piracicaba.sp.gov.br/plantao+coronavirus+02+03+2021.aspx


/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
